In [1]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string


In [50]:


def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    return heapq.nlargest(n, s, key=len)

def get_rarewrd(s):
    s = re.split(' ', s)
    freqs = [words.count(x) for x in s]
    dd= [i for i,x in enumerate(freqs) if x == min(freqs)]
    return [s[dd[0]]]

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A'
		if best == [1]:

			return 'B'
		if best == [2]:
			return 'C'
		if best == [3]:
			return 'D'
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C'
	except:
		print('some error, returning C')
		return 'C'
    
    
    
    
def word_clusters(row):
    if str(row.id)[-2:] == '01':
        print(row.id)
        print('\n\n\n\n')
    print(row.correctAnswer)
    try:
        keywds = row.keyword

        word_pos = {}
        for keyw in keywds:
            ls = [i for i,x in enumerate(words) if x == keyw]
            #print(ls)
            word_pos[keyw] = ls

        spots = []
        for key1 in keywds:
            print(key1)
            matches = {}
            for z in word_pos[key1]:
                dists=[]
                for key_ in keywds:
                    d= abs(min(word_pos[key_], key=lambda x:abs(x-z))-z)
                    dists.append(d)
                #print(z)
                #print(dists)
                dists.remove(max(dists))
                matches[z] = sum(dists)

            zzz= [k for k, v in matches.items() if v ==min(matches.values())]
            spots.append(zzz[0])
        print(spots)
        answers = row['akeyword':'dkeyword']
        kkk = []
        for answer in answers:
            print(answer[0])
            waka = 0
            for x in spots:
                wd = words[x-1000:x+1000]
                ct = wd.count(answer[0])
                waka = (waka + ct)#/(words.count(answer[0])/2)    #trying to skew for 
            kkk.append(waka)
        mx = max(kkk)
        print(kkk)
        best = [i for i, j in enumerate(kkk) if j == mx]
        if best == [0]:
            return 'A'
        if best == [1]:
            return 'B'
        if best == [2]:
            return 'C'
        if best == [3]:
            return 'D'
        else:
            print('returning C becuase no max')
            return 'C'
    except:
        print('returning C becuase of BIG PROBLEM')
        return 'C'
    

In [67]:
data_all  = pd.read_csv('../input/training_set.tsv', '\t')
train = data_all.sample(frac=0.98, random_state=1)
data = data_all.loc[~data_all.index.isin(train.index)]

In [68]:
file = open('../input/Concepts.txt', 'r')
words = list(file.read().split())
words = [word.strip(string.punctuation) for word in words]

In [69]:
data.question = data.question.apply(remove_punctuation)
data.answerA = data.answerA.apply(remove_punctuation)
data.answerB = data.answerB.apply(remove_punctuation)
data.answerC = data.answerC.apply(remove_punctuation)
data.answerD = data.answerD.apply(remove_punctuation)

/Users/liamconnell/anaconda/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [71]:
data['keyword'] =data['question'].apply(get_longword, args = (5,))

#I get to choose this
#data['qvec'] = data.keyword.apply(get_avg_vec) 

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [72]:
data['akeyword'] =data['answerA'].apply(get_rarewrd)
data['bkeyword'] =data['answerB'].apply(get_rarewrd)
data['ckeyword'] =data['answerC'].apply(get_rarewrd)
data['dkeyword'] =data['answerD'].apply(get_rarewrd)


/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [73]:
data['guess'] = data.apply(word_clusters, axis = 1)

B
elements
compound
organic
likely
Which
[1181729, 672725, 672754, 1172651, 672753]
helium
hydrogen
mercury
tin
[0, 27, 0, 1]
B
population
condition
ecosystem
collapse
disorder
[194584, 190213, 193861, 332616, 189825]
thickness
decline
diseases
pest
[0, 0, 2, 0]
C
atmosphere
change
clouds
Which
phase
[315553, 470960, 314973, 470983, 470959]
evaporation
freezing
condensation
melting
[0, 9, 0, 0]
C
activities
contribute
extinction
species
Which
[393814, 221535, 393993, 221941, 394018]
hunting
destruction
restoration
nonnative
[5, 0, 0, 0]
B
chemically
combine
product
carbon
oxygen
[466775, 466774, 696664, 466780, 696659]
plus
plus
equal
less
[0, 0, 0, 6]
A
properties
Magnesium
Magnesium
reactive
periodic
[505047, 505123, 505123, 1169478, 1169551]
lithium
silver
chlorine
nonreactive
[5, 7, 5, 2]
C
projection
classified
vertebral
between
another
[960059, 857838, 858226, 857803, 857802]
ballandsocket
hinge
pivot
sliding
[0, 0, 0, 0]
returning C becuase no max
B
conditions
following
humidity

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [74]:
sum(data.correctAnswer ==data.guess)/len(data) #best was 32 with 2 qwds and 1 aRarewd

0.32000000000000001

In [59]:
data.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD,keyword,akeyword,bkeyword,ckeyword,dkeyword,guess
129,100130,Which phase change in the atmosphere forms clouds,C,evaporation,freezing,condensation,melting,"[atmosphere, change]",[evaporation],[freezing],[condensation],[melting],B
144,100145,Which of these human activities does not contr...,C,hunting,habitat destruction,restoration ecology,introduced nonnative species,"[activities, contribute]",[hunting],[destruction],[restoration],[nonnative],C
235,100236,When carbon and oxygen combine chemically the ...,B,greater than the mass of the carbon plus the m...,equal to the mass of the carbon plus the mass ...,equal to the mass of the carbon,less than the mass of the carbon,"[chemically, combine]",[plus],[plus],[equal],[less],D
252,100253,Magnesium and sodium are highly reactive metal...,A,highly reactive metals like lithium and potassium,less reactive metals like gold and silver,highly reactive nonmetals like chlorine and ox...,almost nonreactive gases like helium and neon,"[properties, Magnesium]",[lithium],[silver],[chlorine],[nonreactive],C
319,100320,High humidity often leads to foggy weather or ...,B,barometer,psychrometer,thermometer,anemometer,"[conditions, following]",[barometer],[psychrometer],[thermometer],[anemometer],C


In [39]:
look = ['developmental', 'physiological', 'processes']

In [40]:
look[0] in words

True

In [43]:
word_pos = {}
for lk in look:
    ls = [i for i,x in enumerate(words) if x == lk]
            #print(ls)
    word_pos[lk] = ls

In [44]:
for key in word_pos:
    print(word_pos[key])

[1193, 1207, 6201, 7045, 18461, 29634, 29727, 53344, 59059, 76247, 78507, 84890, 84967, 85520, 93957, 97624, 110308, 126219, 126972, 135139, 135142, 135226, 135330, 135949, 136900, 138999, 139020, 139197, 151894, 152338, 158102, 158147, 161354, 180498, 204986, 205307, 207131, 209804, 210181, 211519, 215792, 216321, 216900, 219860, 227865, 230199, 242659, 251996, 252682, 252687, 252761, 253363, 257452, 257470, 258535, 261209, 261757, 261799, 261926, 261950, 262189, 262233, 262263, 262387, 262398, 262403, 262408, 262531, 262533, 262546, 272171, 272180, 276192, 276249, 285628, 292802, 295581, 295770, 296097, 296584, 305290, 305371, 305459, 311133, 321081, 323595, 332374, 343066, 343255, 348237, 348293, 348299, 348330, 348336, 348342, 348620, 349051, 349809, 350633, 351107, 351292, 351308, 352938, 359105, 364654, 365098, 367246, 367739, 367789, 369288, 374720, 374875, 376253, 377302, 380090, 381142, 386021, 386658, 396673, 396681, 396688, 396695, 397350, 405330, 418128, 418135, 418603, 418

In [45]:
jkl = [1193, 1207, 6201, 7045, 18461, 29634, 29727, 53344, 59059, 76247, 78507, 84890, 84967, 85520, 93957, 97624, 110308, 126219, 126972, 135139, 135142, 135226, 135330, 135949, 136900, 138999, 139020, 139197, 151894, 152338, 158102, 158147, 161354, 180498, 204986, 205307, 207131, 209804, 210181, 211519, 215792, 216321, 216900, 219860, 227865, 230199, 242659, 251996, 252682, 252687, 252761, 253363, 257452, 257470, 258535, 261209, 261757, 261799, 261926, 261950, 262189, 262233, 262263, 262387, 262398, 262403, 262408, 262531, 262533, 262546, 272171, 272180, 276192, 276249, 285628, 292802, 295581, 295770, 296097, 296584, 305290, 305371, 305459, 311133, 321081, 323595, 332374, 343066, 343255, 348237, 348293, 348299, 348330, 348336, 348342, 348620, 349051, 349809, 350633, 351107, 351292, 351308, 352938, 359105, 364654, 365098, 367246, 367739, 367789, 369288, 374720, 374875, 376253, 377302, 380090, 381142, 386021, 386658, 396673, 396681, 396688, 396695, 397350, 405330, 418128, 418135, 418603, 418610, 446213, 446733, 446934, 468120, 468408, 468687, 474378, 474620, 474648, 474654, 474665, 475055, 476576, 537216, 537655, 539075, 539453, 541086, 541586, 541597, 551886, 632454, 641076, 669398, 670940, 672143, 674043, 677458, 677533, 677767, 679879, 680023, 680083, 680266, 681071, 690199, 690299, 691081, 691095, 694234, 694336, 694383, 694528, 695336, 695381, 695385, 695711, 695749, 696518, 696528, 696555, 696916, 697147, 698066, 707809, 707916, 713070, 713240, 713681, 723581, 729721, 732227, 732616, 734092, 742503, 747728, 748340, 748511, 753508, 761030, 761365, 761686, 761723, 762132, 762213, 762288, 762713, 762774, 774272, 776463, 776801, 777714, 782376, 822854, 822962, 823039, 859376, 860031, 860438, 860854, 863506, 863701, 863865, 864298, 870712, 870732, 871101, 871155, 874530, 874884, 875597, 875635, 875686, 877927, 880863, 883315, 884027, 885762, 886323, 886416, 896083, 903476, 917980, 918415, 924537, 928924, 935015, 935112, 935321, 937543, 941087, 941918, 942341, 944160, 945879, 945961, 946094, 946522, 946710, 947691, 947704, 952094, 952595, 955454, 955830, 957259, 962044, 969014, 969208, 970237, 970251, 971013, 974466, 974902, 975451, 977545, 978052, 978373, 979693, 980240, 981098, 983060, 985205, 985252, 986079, 986923, 987356, 988499, 989170, 989188, 990926, 991927, 992094, 994230, 999752, 1001690, 1001695, 1001891, 1004513, 1006596, 1009534, 1011313, 1011322, 1012602, 1017388, 1018953, 1019107, 1019163, 1019619, 1019719, 1021624, 1025794, 1026301, 1026654, 1026697, 1028329, 1059474, 1066194, 1066711, 1067116, 1070224, 1071343, 1076067, 1078657, 1078834, 1079396, 1081984, 1082038, 1082118, 1082146, 1082156, 1082399, 1082677, 1088024, 1088082, 1090399, 1094537, 1096429, 1104813, 1107845, 1108914, 1109605, 1109616, 1109810, 1109903, 1110501, 1110627, 1111809, 1111931, 1113843, 1114433, 1123021, 1123949, 1128763, 1137859, 1144002, 1178006, 1179459, 1180362, 1209997, 1217061, 1219493, 1247602, 1249437, 1249855, 1257413, 1257853, 1268027, 1268077, 1269847, 1271987, 1272656, 1275367, 1288618, 1291502, 1292933, 1294457, 1311565, 1311672, 1311846, 1311893, 1312721, 1312815, 1312872, 1313345, 1313514, 1314049, 1314110, 1314282, 1315850, 1316268, 1317458, 1318055, 1319059, 1319073, 1323716, 1324331, 1324527, 1326224, 1326718, 1328412, 1328471, 1328507, 1328562, 1328630, 1329427, 1329877, 1329952, 1346925, 1350780, 1378099, 1421275, 1421802, 1535753]

In [57]:
zz = [93956, 110307, 678781, 925610, 931381, 947579, 987988, 1346204, 1535749]
for z in zz:
    print(min(jkl, key=lambda x:abs(x-z))-z)

1
1
-1014
-1073
-2457
112
511
721
4


In [54]:
words[110300:110317]

['used',
 'in',
 'scientific',
 'research',
 'especially',
 'developmental',
 'and',
 'physiological',
 'processes',
 'largely',
 'due',
 'to',
 'their',
 'unique',
 'ability',
 'to',
 'undergo']